# How to escape from the mystical forest?

##### The problem:

In a mystical forest, there is a series of enchanted stepping stones and treacherous quicksand pits. A brave adventurer must navigate through this forest to reach a hidden treasure. The forest has a linear path with multiple sections, each section consisting of either a stepping stone or quicksand. The goal is to find the minimum number of moves and the exact path to reach the treasure.

For example, consider the forest represented by the following sequence:
-	[1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1] where 1 means stepping stones and 0 means quicksand, the length of this specific example is 18 units.

The adventurer can leap exactly 1, 2, 3 or 5 units forward.

Compute the minimum number of moves required and determine the precise path to successfully navigate the forest based on the provided representation!

##### Examples:

-	[1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1] - Solution: 2  moves, Path: [1, 6, 11]
-	[1, 1, 0, 1, 0, 1, 1] - Solution: 2  moves, Path: [1, 6, 7] 
-	[1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1] - Solution: No solution, Path: [1]
-	[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1] - Solution: 3  moves, Path: [1, 4, 7, 11] 

##### Limitations:

-	The forest representation always starts and ends with a 1. The first element represents the starting point, and the last element represents the endpoint.
-	If the adventurer lands on a stepping stone that allows them to jump beyond the endpoint, it is considered a valid solution (see example 4).
-	If there is no valid path to reach the treasure, the path should be: Solution: No solution, Path: [1] 


In [9]:
import pandas as pd
import numpy as np
import random

## Let's create some test forests of varying lengths

In [10]:
# Creating an empty list for the results
test_forests = []
random.seed(10)

# makelist is a list comprehension to be used in main, it gives us the lava pools and marble platforms of the forest.
def makelist(count):
    return [random.randint(0, 1) for _ in range(count)]

# main creates the forests
def main():
    random_integer = random.randint(6, 29)  # Generate a random integer where 5 < i < 30. It sets the length of the forest.
    forest = makelist(count=random_integer)  # Make a list
    forest[0] = 1 # Adhering to the limitation of the forest starting with a stepping stone
    forest[-1] = 1 # Adhering to the limitation of the forest ending with a stepping stone
    return forest  # Return the list of 0s and 1s
    

# We use function main 50 times to create a list of lists (test_forests) containing 30 forests
for i in range(50):
    test_forests.extend([main()])

print(test_forests)

[[1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1], [1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1], [1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1], [1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1], [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1], [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1], [1, 1, 1, 0, 1, 1], [1, 1, 0, 1, 0, 1, 1], [1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1], [1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1], [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 0, 1], [1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1], [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1], [1, 1, 0, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0

## The function that solves the puzzle

In [11]:
def escape_forest(forest):
    """Calculates the number of moves and the exact path the adventurer needs to take to get to the treasure.
    
    Rules of the game:
    - A forest is a list of 0s and 1s where the 1s mean stepping stones and 0s mean deadly quicksands.
    - A forest always starts and ends with a 1.
    - The adventurer can only stand on stepping stones.
    - The adventurer can jump 1, 2, 3, or 5 steps forward, he/she just needs to start and finish on a marble platform.
    - The adventurer can reach the treasure by teleporting beyond the end of the forest as well.
    
    Parameters:
    - forest(list): A list of 0s and 1s
    
    Returns:
    - If the adventurer can escape the forest the function returns the stepping stones used and the number of steps taken
    - If the adventurer cannot escape the forest the function returns "Solution: No solution, Path: [1]".
    """
    
    n = len(forest)
    moves = [float('inf')] * n  # Initialize an array with infinity values (to act as unbounded upper values for comparison)
    moves[0] = 0  # First element doesn't require any move

    # Update the array at each "stepping stone" with the minimum amount of moves to get there
    for i in range(1, n):
        if forest[i] == 1 and i != n - 1:
            for j in [1, 2, 3, 5]:
                if i - j >= 0:
                    moves[i] = min(moves[i], moves[i - j] + 1)
        # For the end of the forest we let the function use the distance of 4 as well to let the knight arrive beyond the end
        if forest[i] == 1 and i == n - 1:
            for j in [1, 2, 3, 4, 5]:
                if i - j >= 0:
                    moves[i] = min(moves[i], moves[i - j] + 1)

    # Return "No solution" if the array's last element wasn't updated to something smaller than infinity
    if moves[n - 1] == float('inf'):
        return "Solution: No solution, Path: [1]"
    # Retrace the moves from the endpoint of the array
    else:
        # Trace the path of moves
        path = [n - 1]
        curr_index = n - 1
        while curr_index != 0:
            if curr_index == n - 1:
                for j in [1, 2, 3, 4, 5]:
                    # Looking for the first element that is smaller by exactly 1 in our moves array jumping/teleporting backwards
                    if curr_index - j >= 0 and moves[curr_index - j] + 1 == moves[curr_index]:
                        path.append(curr_index - j)
                        curr_index -= j
                        break
            if curr_index != n - 1:
                for j in [1, 2, 3, 5]:
                    if curr_index - j >= 0 and moves[curr_index - j] + 1 == moves[curr_index]:
                        path.append(curr_index - j)
                        curr_index -= j
                        break

        path.reverse()  # Reverse the path to get the correct order
        return path


## Let's test our function by running it on all our test forests

In [12]:
for i in range(len(test_forests)):
    forest = test_forests[i]
    print("Test forest", i+1)
    print(test_forests[i])
    result = escape_forest(forest)
    
    if result == "Solution: No solution, Path: [1]":
        print(result, "\n")
    else:
        # As the program works with 0 indexed arrays, the path numbers must be increased by 1
        print("Solution:", len(result) - 1, " moves,", "Path:", [i + 1 for i in result], "\n")
        

Test forest 1
[1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]
Solution: 6  moves, Path: [1, 6, 11, 12, 17, 22, 24] 

Test forest 2
[1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1]
Solution: 2  moves, Path: [1, 6, 11] 

Test forest 3
[1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1]
Solution: 3  moves, Path: [1, 6, 9, 12] 

Test forest 4
[1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1]
Solution: 3  moves, Path: [1, 4, 9, 13] 

Test forest 5
[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1]
Solution: 3  moves, Path: [1, 4, 7, 11] 

Test forest 6
[1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1]
Solution: 5  moves, Path: [1, 6, 8, 13, 18, 20] 

Test forest 7
[1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1]
Solution: 5  moves, Path: [1, 3, 8, 13, 18, 21] 

Test forest 8
[1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1]
Solution: 6  moves, Path: [1, 3, 8, 13, 15, 20, 23] 

Test forest 9
[1, 1, 1, 0, 1, 1]
Solution: 1  moves, Path: [1, 6] 

Test forest 10
[1, 1, 0, 1, 0, 1,

#### The function seems to work flawlessly.